### RIR Watermark Experiment
- Test the quality of clean speech and rir
- Test the synthesized re-echoed speech
- Test decoding accuracy :BER

In [1]:
import sys, torch, torchaudio
sys.path.append("/home/c3-server1/Documents/Chenpei/re-rir")
from model import *
from data.util import unpickle_data
from dataloader import RIR_Dataset, create_dataloader

val_entries   = unpickle_data("../data/synthetic/test-real-rir_real_2.pkl")
va_ds = RIR_Dataset(val_entries)

_, va_dl = create_dataloader(va_ds, va_ds, batch_size=4, num_workers=16)

rs, clean, rir = next(iter(va_dl))

/home/c3-server1/anaconda3/envs/re-rir312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/c3-server1/Documents/Chenpei/re-rir/ReEcho_Net/dataloader.py:50: UserWarning: torchaudio._backend.utils.load has been deprecated. As TorchAudio is no longer being actively developed, this function can no longer be supported.See https://github.com/pytorch/audio/issues/3902 for more details. It will be removed from 2.9 release. 
  audio, sr = torchaudio.load(file_path)
/home/c3-server1/anaconda3/envs/re-rir312/lib/python3.12/site-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.__init__ has been deprecated. As TorchAudio is no longer being actively developed, this function can no longer be supported.See https://github.com/pytorch/audio/issues/3902 for more details. It wi

In [2]:
device = "cpu"

separator = ReEcho_Separator()
generator = ReEcho_Generator()
watermarker = ReEcho_WM(msg_len=5)
spec_transform = SpectrogramTransform()

model = torch.load("checkpoints/0711_0906/rir_model_epoch_220.pth")

# Load model parameters
separator.load_state_dict(model['separator_state_dict'])
generator.load_state_dict(model['generator_state_dict'])
watermarker.load_state_dict(model['watermarker_state_dict'])

separator.eval()
generator.eval()
watermarker.eval()

/home/c3-server1/anaconda3/envs/re-rir312/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


ReEcho_WM(
  (wm_net): WMEmbedderExtractor(
    (msg_codebook): Embedding(32, 128)
    (embedder): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Swish(
        (silu): SiLU()
      )
      (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (encoder): ConformerEncoder(
      (input_proj): Sequential(
        (0): Linear(in_features=513, out_features=256, bias=True)
        (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (2): Swish(
          (silu): SiLU()
        )
      )
      (encoder): ConformerEncoder(
        (layers): ModuleList(
          (0-11): 12 x ConformerEncoderLayer(
            (mha_layer): RelPosMHAXL(
              (dropout_att): Dropout(p=0.1, inplace=False)
              (out_proj): Linear(in_features=256, out_features=256, bias=True)
              (linear_pos): Linear(in_features=256, out_features=256, bias=False)
            )
            (convolution_module): ConvolutionModule(
        

In [8]:
import torchaudio.functional as F

with torch.no_grad():
    msg = torch.randint(0, 2, (rs.shape[0], watermarker.msg_len), device=device)

    # feature extraction
    spec_masked, rir_emb = separator(rs)

    # watermark embedding 
    rir_emb_wm = watermarker.embedding(msg, rir_emb)

    # generation
    rir_est = generator(rir_emb_wm)

    idx = torch.randperm(clean.shape[0])
    clean_permute = clean[idx]
    rs_synthesize = F.fftconvolve(clean_permute, rir_est, mode='full')
    rs_synthesize = rs_synthesize[:, :rs.shape[1]]


    # watermark extraction
    rs_resyn_spec = spec_transform(rs_synthesize)
    msg_decoded = watermarker.extraction(rs_resyn_spec, mode='msg')

    print(msg_decoded)
    print(msg)
    print(msg_decoded == msg)

tensor([[1., 1., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 1., 1., 0.],
        [0., 1., 1., 1., 0.]])
tensor([[1, 1, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 1, 1, 0],
        [0, 1, 1, 1, 0]])
tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])


Now, show the results of task 1:

In [7]:
from IPython.display import Audio, display
from speechbrain.inference.vocoders import HIFIGAN
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-libritts-16kHz", savedir="pretrained_models/tts-hifigan-libritts-16kHz")

mel_transform = MelScaleTransform()
dereverb = hifi_gan.decode_batch(mel_transform(spec_masked))

print("clean")
display(Audio(clean[0].numpy(), rate=16000))
print("rs")
display(Audio(rs[0].numpy(), rate=16000))
print("rs_resyn")
display(Audio(rs_resyn[0].numpy(), rate=16000))
print("dereverb")
display(Audio(dereverb[0].numpy(), rate=16000))

/home/c3-server1/anaconda3/envs/re-rir312/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


clean


rs


rs_resyn


NameError: name 'rs_resyn' is not defined